<a href="https://colab.research.google.com/github/QROST/AlpacaPlayground/blob/main/AlpacaTFSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpaca-trade-api
!pip install --upgrade mplfinance
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

     |████████████████████████████████| 81kB 2.9MB/s 
     |████████████████████████████████| 204kB 4.7MB/s 
     |████████████████████████████████| 61kB 1.9MB/s 
Selecting previously unselected package libta-lib0.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 2.7MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437793 sha256=47e194b7d794f73c191e987d629bb4c61f299b20c2ace81

In [ ]:
from datetime import datetime
import numpy as np
import talib
import alpaca_trade_api as tradeapi
import pandas
from time import sleep
import os


# Creates dataset folders in directory script is run from
try:
    os.stat("./train")
    os.stat("./eval")
except BaseException:
    os.mkdir("./train")
    os.mkdir("./eval")



In [ ]:
api = tradeapi.REST(
    "PKY2N25P0N57TF83ZKXS",
    "wycEE2hPm0C2OOzyigVvikT4IZEboOMqTeDKOQhT")

In [ ]:

# api = tradeapi.REST(key_id= < your key id >, secret_key= < your secret
# key > )

barTimeframe = "1D"  # 1Min, 5Min, 15Min, 1H, 1D

assetList = ['NVDA','MSFT']


In [ ]:

# ISO8601 date format
trainStartDate = "2015-01-01T00:00:00.000Z"
trainEndDate = "2017-06-01T00:00:00.000Z"
evalStartDate = "2017-06-01T00:00:00.000Z"
evalEndDate = "2020-06-01T00:00:00.000Z"

targetLookaheadPeriod = 1
startCutoffPeriod = 50  # Set to length of maximum period indicator


# Tracks position in list of symbols to download
iteratorPos = 0
assetListLen = len(assetList)


In [ ]:
symbol = assetList[iteratorPos]

# Returns market data as a pandas dataframe
returned_data = api.get_barset(
    symbol,
    barTimeframe,
    start=trainStartDate,
    end=evalEndDate).df

<bound method NDFrame.first of                              NVDA                                    
                             open    high      low    close    volume
time                                                                 
2014-12-31 00:00:00-05:00   20.40   20.51   19.990   20.050   3487614
2015-01-02 00:00:00-05:00   20.13   20.28   19.811   20.125   2416481
2015-01-05 00:00:00-05:00   20.13   20.19   19.700   19.800   3720891
2015-01-06 00:00:00-05:00   19.82   19.84   19.170   19.180   4393047
2015-01-07 00:00:00-05:00   19.33   19.50   19.080   19.140   5829847
...                           ...     ...      ...      ...       ...
2020-05-22 00:00:00-04:00  353.01  363.72  348.530  361.060  24510192
2020-05-26 00:00:00-04:00  366.27  367.27  346.880  348.780  17690647
2020-05-27 00:00:00-04:00  345.00  345.31  319.870  341.130  28411380
2020-05-28 00:00:00-04:00  336.49  350.42  335.170  339.310  35425530
2020-05-29 00:00:00-04:00  342.18  354.67  339.400  353.650

In [ ]:
# Processes all data into numpy arrays for use by talib
timeList = np.array(returned_data.index)
openList = np.array(returned_data[symbol]['open'].tolist(), dtype=np.float64)
print(openList)
#print(returned_data)


[ 20.4   20.13  20.13 ... 345.   336.49 342.18]


In [ ]:
highList = np.array(returned_data[symbol]['high'].tolist(), dtype=np.float64)
lowList = np.array(returned_data[symbol]['low'].tolist(), dtype=np.float64)
closeList = np.array(returned_data[symbol]['close'].tolist(), dtype=np.float64)
volumeList = np.array(returned_data[symbol]['volume'].tolist(), dtype=np.float64)

In [ ]:
# Adjusts data lists due to the reward function look ahead period
shiftedTimeList = timeList[:-targetLookaheadPeriod]
shiftedClose = closeList[targetLookaheadPeriod:]
highList = highList[:-targetLookaheadPeriod]
lowList = lowList[:-targetLookaheadPeriod]
closeList = closeList[:-targetLookaheadPeriod]

In [ ]:
# Calculate trading indicators
RSI14 = talib.RSI(closeList, 14)
RSI50 = talib.RSI(closeList, 50)
STOCH14K, STOCH14D = talib.STOCH(
    highList, lowList, closeList, fastk_period=14, slowk_period=3, slowd_period=3)


In [ ]:
# Calulate network target/ reward function for training
closeDifference = shiftedClose - closeList
closeDifferenceLen = len(closeDifference)


In [ ]:

# Creates a binary output if the market moves up or down, for use as
# one-hot labels
longOutput = np.zeros(closeDifferenceLen)
longOutput[closeDifference >= 0] = 1
shortOutput = np.zeros(closeDifferenceLen)
shortOutput[closeDifference < 0] = 1


In [ ]:

# Constructs the dataframe and writes to CSV file
outputDF = {
    "close": closeList,  # Not to be included in network training, only for later analysis
    "RSI14": RSI14,
    "RSI50": RSI50,
    "STOCH14K": STOCH14K,
    "STOCH14D": STOCH14D,
    "longOutput": longOutput,
    "shortOutput": shortOutput
}


In [ ]:

# Makes sure the dataframe columns don't get mixed up
columnOrder = ["close", "RSI14", "RSI50", "STOCH14K",
                "STOCH14D", "longOutput", "shortOutput"]
outputDF = pandas.DataFrame(
    data=outputDF,
    index=shiftedTimeList,
    columns=columnOrder)[
    startCutoffPeriod:]


ValueError: ignored

In [ ]:

# Splits data into training and evaluation sets
trainingDF = outputDF[outputDF.index < evalStartDate]
evalDF = outputDF[outputDF.index >= evalStartDate]

if (len(trainingDF) > 0 and len(evalDF) > 0):
    print("writing " + str(symbol) +
          ", data len: " + str(len(closeList)))

    trainingDF.to_csv("./train/" + symbol + ".csv", index_label="date")
    evalDF.to_csv("./eval/" + symbol + ".csv", index_label="date")    

In [ ]:
# Splits data into training and evaluation sets
        trainingDF = outputDF[outputDF.index < evalStartDate]
        evalDF = outputDF[outputDF.index >= evalStartDate]

        if (len(trainingDF) > 0 and len(evalDF) > 0):
            print("writing " + str(symbol) +
                  ", data len: " + str(len(closeList)))

            trainingDF.to_csv("./train/" + symbol + ".csv", index_label="date")
            evalDF.to_csv("./eval/" + symbol + ".csv", index_label="date")

2

In [ ]:
import argparse
import sys
import tempfile
from time import time
import random
from os import listdir
from os.path import isfile, join


import pandas
import numpy as np
import tensorflow as tf
from sklearn import metrics



In [ ]:

# model settings
# Static seed to allow for reproducability between training runs
tf.set_random_seed(12345)
trainingCycles = 500000  # Number of training steps before ending
batchSize = 1000  # Number of examples per training batch
summarySteps = 1000  # Number of training steps between each summary
dropout = 0.5  # Node dropout for training
nodeLayout = [40, 30, 20, 10]  # Layout of nodes in each layer


mainDirectory = str("./model_1/")

trainFiles = [f for f in listdir("./train/") if isfile(join("./train/", f))]
evalFiles = [f for f in listdir("./eval/") if isfile(join("./eval/", f))]


# Initialises data arrays
trainDataX = np.empty([0, 4])
trainDataY = np.empty([0, 2])
evalDataX = np.empty([0, 4])
evalDataY = np.empty([0, 2])


# Reads training data into memory
readPos = 0
for fileName in trainFiles:
    importedData = pandas.read_csv("./train/" + fileName, sep=',')

    xValuesDF = importedData[["RSI14", "RSI50", "STOCH14K", "STOCH14D"]]
    yValuesDF = importedData[["longOutput", "shortOutput"]]

    xValues = np.array(xValuesDF.values.tolist())
    yValues = np.array(yValuesDF.values.tolist())

    trainDataX = np.concatenate([trainDataX, xValues], axis=0)
    trainDataY = np.concatenate([trainDataY, yValues], axis=0)

    if readPos % 50 == 0 and readPos > 0:
        print("Loaded " + str(readPos) + " training files")

    readPos += 1
print("\n\n")


# Reads evalutation data into memory
readPos = 0
for fileName in evalFiles:
    importedData = pandas.read_csv("./eval/" + fileName, sep=',')

    xValuesDF = importedData[["RSI14", "RSI50", "STOCH14K", "STOCH14D"]]
    yValuesDF = importedData[["longOutput", "shortOutput"]]

    xValues = np.array(xValuesDF.values.tolist())
    yValues = np.array(yValuesDF.values.tolist())

    evalDataX = np.concatenate([evalDataX, xValues], axis=0)
    evalDataY = np.concatenate([evalDataY, yValues], axis=0)

    if readPos % 50 == 0 and readPos > 0:
        print("Loaded " + str(readPos) + " training files")

    readPos += 1
print("\n\n")


# used to sample batches from your data for training
def createTrainingBatch(amount):

    randomBatchPos = np.random.randint(0, trainDataX.shape[0], amount)

    xOut = trainDataX[randomBatchPos]
    yOut = trainDataY[randomBatchPos]

    return xOut, yOut


tf.logging.set_verbosity(tf.logging.INFO)

# ML training and evaluation functions
def train():
    globalStepTensor = tf.Variable(0, trainable=False, name='global_step')

    sess = tf.InteractiveSession()

    # placeholder for the input features
    x = tf.placeholder(tf.float32, [None, 4])
    # placeholder for the one-hot labels
    y = tf.placeholder(tf.float32, [None, 2])
    # placeholder for node dropout rate
    internalDropout = tf.placeholder(tf.float32, None)

    net = x  # input layer is the trading indicators

    # Creates the neural network model
    with tf.name_scope('network'):
        # Initialises each layer in the network
        layerPos = 0
        for units in nodeLayout:
            net = tf.layers.dense(
                net,
                units=units,
                activation=tf.nn.tanh,
                name=str(
                    "dense" +
                    str(units) +
                    "_" +
                    str(layerPos)))  # adds each layer to the networm as specified by nodeLayout
            # dropout layer after each layer
            net = tf.layers.dropout(net, rate=internalDropout)
            layerPos += 1

    logits = tf.layers.dense(
        net, 2, activation=tf.nn.softmax)  # network output

    with tf.name_scope('lossFunction'):
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(
                labels=y,
                logits=logits))  # on NO account put this within a name scope - tensorboard shits itself

    with tf.name_scope('trainingStep'):
        tf.summary.scalar('crossEntropyLoss', cross_entropy_loss)
        trainStep = tf.train.AdamOptimizer(0.0001).minimize(
            cross_entropy_loss, global_step=globalStepTensor)

    with tf.name_scope('accuracy'):
        correctPrediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge_all()
    trainWriter = tf.summary.FileWriter(
        mainDirectory + '/train', sess.graph, flush_secs=1, max_queue=2)
    evalWriter = tf.summary.FileWriter(
        mainDirectory + '/eval', sess.graph, flush_secs=1, max_queue=2)
    tf.global_variables_initializer().run()

    # Saves the model at defined checkpoints and loads any available model at
    # start-up
    saver = tf.train.Saver(max_to_keep=2, name="checkpoint")
    path = tf.train.get_checkpoint_state(mainDirectory)
    if path is not None:
        saver.restore(sess, tf.train.latest_checkpoint(mainDirectory))

    lastTime = time()
    while tf.train.global_step(sess, globalStepTensor) <= trainingCycles:
        globalStep = tf.train.global_step(sess, globalStepTensor)

        # generates batch for each training cycle
        xFeed, yFeed = createTrainingBatch(batchSize)

        # Record summaries and accuracy on both train and eval data
        if globalStep % summarySteps == 0:
            currentTime = time()
            totalTime = (currentTime - lastTime)
            print(str(totalTime) + " seconds, " +
                  str(summarySteps / totalTime) + " steps/sec")
            lastTime = currentTime

            summary, accuracyOut, _ = sess.run([
                merged,
                accuracy,
                trainStep
            ],
                feed_dict={
                    x: xFeed,
                    y: yFeed,
                    internalDropout: dropout
                })
            trainWriter.add_summary(summary, globalStep)
            trainWriter.flush()
            print('Train accuracy at step %s: %s' % (globalStep, accuracyOut))

            summary, accuracyOut = sess.run([
                merged,
                accuracy,
            ],
                feed_dict={
                    x: evalDataX,
                    y: evalDataY,
                    internalDropout: 0
                })
            evalWriter.add_summary(summary, globalStep)
            evalWriter.flush()
            print('Eval accuracy at step %s: %s' % (globalStep, accuracyOut))

            print("\n\n")
            saver.save(sess, save_path=str(mainDirectory + "model"),
                       global_step=globalStep)  # saves a snapshot of the model

        else:  # Training cycle
            _ = sess.run(
                [trainStep], feed_dict={
                    x: xFeed,
                    y: yFeed,
                    internalDropout: dropout
                })

    trainWriter.close()
    evalWriter.close()


train()